In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import h5py
import cv2
import os
import h5py
from PIL import Image
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense, Input,concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout
from keras.models import Model
from keras.models import Sequential
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from sklearn.metrics import pairwise_distances

In [ ]:
# Diretório onde estão as imagens
base_dir = '/content/drive/MyDrive/post-processed'

# Lista para armazenar imagens e rótulos
images = []
labels = []

# Tamanho desejado para as imagens
target_size = (224, 224)

# Iterar sobre as pastas (uma para cada pessoa)
for person_folder in os.listdir(base_dir):
    person_path = os.path.join(base_dir, person_folder)

    # Iterar sobre as imagens em cada pasta
    for filename in os.listdir(person_path):
        img_path = os.path.join(person_path, filename)

        # Carregar a imagem e redimensionar para o tamanho desejado
        print(img_path)
        img = cv2.imread(img_path)
        if img is not None:
          img = cv2.resize(img, target_size)

          # Adicionar a imagem à lista
          images.append(img)

          # Adicionar o rótulo correspondente (assumindo que o nome da pasta é o rótulo)
          labels.append(person_folder)
        else:
          print("Erro ao carregar a imagem:", img_path)
# Converter listas em arrays numpy
images = np.array(images)
labels = np.array(labels)

/content/drive/MyDrive/post-processed/Michel_Therrien/Michel_Therrien_0004_0000.jpg
/content/drive/MyDrive/post-processed/Michel_Therrien/Michel_Therrien_0001_0001.jpg
/content/drive/MyDrive/post-processed/Michel_Therrien/Michel_Therrien_0001_0000.jpg
/content/drive/MyDrive/post-processed/Michel_Temer/Michel_Temer_0002_0001.jpg
/content/drive/MyDrive/post-processed/Michel_Temer/Michel_Temer_0002_0000.jpg
/content/drive/MyDrive/post-processed/Michel_Temer/Michel_Temer_0001_0000.jpg
/content/drive/MyDrive/post-processed/Michel_Temer/Michel_Temer_0001_0001.jpg
/content/drive/MyDrive/post-processed/Michelle_Yeoh/Michelle_Yeoh_0004_0000.jpg
/content/drive/MyDrive/post-processed/Michelle_Yeoh/Michelle_Yeoh_0001_0000.jpg
/content/drive/MyDrive/post-processed/Michel_Duclos/Michel_Duclos_0002_0000.jpg
/content/drive/MyDrive/post-processed/Michel_Duclos/Michel_Duclos_0001_0000.jpg
/content/drive/MyDrive/post-processed/Miguel_Contreras/Miguel_Contreras_0001_0000.jpg
/content/drive/MyDrive/post-pr

KeyboardInterrupt: ignored

In [ ]:
# Converter rótulos em codificação one-hot
label_dict = {label: idx for idx, label in enumerate(np.unique(labels))}
labels_encoded = np.vectorize(label_dict.get)(labels)
labels_one_hot = to_categorical(labels_encoded)


num_classes = len(np.unique(labels))
# Dividir dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)

np.save('/content/drive/MyDrive/train_test/X_train.npy', X_train)
np.save('/content/drive/MyDrive/train_test/X_test.npy', X_test)
np.save('/content/drive/MyDrive/train_test/y_train.npy', y_train)
np.save('/content/drive/MyDrive/train_test/y_test.npy', y_test)

# Imprimir as dimensões dos conjuntos de treinamento e teste
print("Dimensões dos conjuntos de treinamento e teste:")
print("Treinamento - Imagens:", X_train.shape, " Rótulos:", y_train.shape)
print("Teste - Imagens:", X_test.shape, " Rótulos:", y_test.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Criar um modelo sequencial
model = Sequential()

# Camadas convolucionais e de pooling para aprender características da imagem
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
# Achatamento para entrada em camadas densas
model.add(Flatten())

# Camadas densas (totalmente conectadas) para fazer a classificação
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))  # Adicionando dropout para evitar overfitting
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.5))

X_train_carregado = np.load('/content/drive/MyDrive/train_test/X_train.npy')
X_test_carregado = np.load('/content/drive/MyDrive/train_test/X_test.npy')
y_train_carregado = np.load('/content/drive/MyDrive/train_test/y_train.npy')
y_test_carregado = np.load('/content/drive/MyDrive/train_test/y_test.npy')

#num_classes = len(np.unique(np.argmax(y_train_carregado, axis=1)))
num_classes = y_train_carregado.shape[1]
# Camada de saída com ativação softmax para classificação
model.add(Dense(num_classes, activation='softmax'))

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



# Treinar o modelo usando X_train e y_train
model.fit(X_train_carregado, y_train_carregado, epochs=45, batch_size=32, validation_split=0.3)

# Avaliar o desempenho do modelo usando X_test e y_test
loss, accuracy = model.evaluate(X_test_carregado, y_test_carregado)

# Resumo do modelo para ver a arquitetura
model.summary()
model.save('/content/drive/MyDrive/train_test/modelo_treinado.h5')


Epoch 1/45
204/204 [==============================] - 22s 77ms/step - loss: 8.1284 - accuracy: 0.0017 - val_loss: 7.9412 - val_accuracy: 0.0018
Epoch 2/45
204/204 [==============================] - 15s 72ms/step - loss: 7.7919 - accuracy: 0.0023 - val_loss: 7.9709 - val_accuracy: 0.0018
Epoch 3/45
204/204 [==============================] - 14s 67ms/step - loss: 7.7214 - accuracy: 0.0023 - val_loss: 8.0353 - val_accuracy: 0.0018
Epoch 4/45
204/204 [==============================] - 14s 68ms/step - loss: 7.7003 - accuracy: 0.0034 - val_loss: 8.0900 - val_accuracy: 0.0018
Epoch 5/45
204/204 [==============================] - 14s 67ms/step - loss: 7.6854 - accuracy: 0.0037 - val_loss: 8.0871 - val_accuracy: 0.0043
Epoch 6/45
204/204 [==============================] - 15s 72ms/step - loss: 7.5945 - accuracy: 0.0077 - val_loss: 8.1065 - val_accuracy: 0.0090
Epoch 7/45
204/204 [==============================] - 15s 73ms/step - loss: 7.2360 - accuracy: 0.0276 - val_loss: 7.9121 - val_accuracy:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import cv2

def preprocess_image(image_path):
    # Carregar a imagem usando o OpenCVA
    img = cv2.imread(image_path)
    # Redimensionar a imagem para o tamanho esperado pelo modelo (224x224 no exemplo da VGG)
    img = cv2.resize(img, (224, 224))
    # Converter de BGR para RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Converter para array numpy
    img_array = image.img_to_array(img)
    # Adicionar uma dimensão extra para se adequar à entrada do modelo
    img_array = np.expand_dims(img_array, axis=0)
    # Pré-processar a imagem conforme esperado pela VGG
    img_array = preprocess_input(img_array)
    return img_array

In [ ]:
def extract_features(img_path, model):
    """ Extrai as características de uma imagem usando o modelo VGG16. """
    img_array = preprocess_image(img_path)
    features = model.predict(img_array)
    return features.flatten()

def extract_features_without_predict(img_path):
    """ Extrai as características de uma imagem usando o modelo VGG16. """
    img_array = preprocess_image(img_path)
    return img_array.flatten()

In [ ]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/train_test/modelo_treinado.h5')

In [ ]:
import pickle

base_dir = '/content/drive/MyDrive/post-processed'
# Salvar características e imagens em um banco de dados (arquivo pickle)
feature_database = {}

for person_folder in os.listdir(base_dir):
    person_path = os.path.join(base_dir, person_folder)

    # Iterar sobre as imagens em cada pasta
    for filename in os.listdir(person_path):
        img_path = os.path.join(person_path, filename)
        print(person_path)
        feature_database[img_path] = extract_features(os.path.join(base_dir, img_path), model)

with open('/content/drive/MyDrive/feature_database.pkl', 'wb') as f:
    pickle.dump(feature_database, f)

/content/drive/MyDrive/post-processed/Gina_Centrello
1/1 [==============================] - 7s 7s/step
/content/drive/MyDrive/post-processed/Gina_Centrello
1/1 [==============================] - 0s 18ms/step
/content/drive/MyDrive/post-processed/Gina_Centrello
1/1 [==============================] - 0s 20ms/step
/content/drive/MyDrive/post-processed/Gina_Centrello
1/1 [==============================] - 0s 18ms/step
/content/drive/MyDrive/post-processed/Gina_Gershon
1/1 [==============================] - 0s 18ms/step
/content/drive/MyDrive/post-processed/Gina_Gershon
1/1 [==============================] - 0s 18ms/step
/content/drive/MyDrive/post-processed/Gina_Gershon
1/1 [==============================] - 0s 17ms/step
/content/drive/MyDrive/post-processed/Gina_Gershon
1/1 [==============================] - 0s 27ms/step
/content/drive/MyDrive/post-processed/Gina_Gershon
1/1 [==============================] - 0s 25ms/step
/content/drive/MyDrive/post-processed/Gina_Gershon
1/1 [===========

KeyboardInterrupt: ignored

In [ ]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity


def predict_celebrity_from_db(image_path, db_features):
    # Extrair características da imagem fornecida
    input_features = extract_features_without_predict(image_path)

    # Carregar o banco de dados de características (assumindo que está em um arquivo pickle)


    # Calcular a similaridade de cosseno entre a imagem fornecida e cada imagem no banco de dados
    max_similarity = 0
    predicted_celebrity = None
    predict_class = None
    for celebrity, db_feature in db_features.items():
        similarity = cosine_similarity([input_features], [db_feature])[0][0]
        if similarity > max_similarity:
            max_similarity = similarity
            predicted_celebrity = celebrity

    return predicted_celebrity.split("\\")[-2]

In [ ]:
pessoas = [
    "/content/drive/MyDrive/post-processed/Aaron_Eckhart/Aaron_Eckhart_0001_0000.jpg",
    "/content/drive/MyDrive/post-processed/Aaron_Guiel/Aaron_Guiel_0001_0000.jpg",
    "/content/drive/MyDrive/post-processed/Abbas_Kiarostami/Abbas_Kiarostami_0001_0000.jpg",
]

db_features = None
feature_database = '/content/drive/MyDrive/feature_database.pkl'
with open(feature_database, 'rb') as f:
    db_features = pickle.load(f)
for pessoa in pessoas:
  celebrity = predict_celebrity_from_db(pessoa, db_features)
  print(pessoa + " -->> " + celebrity)

FileNotFoundError: ignored